In [1]:
data_path="C:\eclipse-jee-luna-SR1-win32\workspace\DateYourData_25022016\original data\\"

In [2]:
import pandas as pd
import numpy as np

In [3]:
internDF = pd.read_csv(data_path+'Internship.csv')
studentDF = pd.read_csv(data_path+'Student.csv')
trainDF = pd.read_csv(data_path+'train.csv')
testDF = pd.read_csv(data_path+'test.csv')

In [4]:
studentDF['Num_Exp']=1
len(studentDF.index)

151191

In [5]:
studentDF['S_Date']=pd.to_datetime(studentDF['Start Date'], format='%d-%m-%Y')
studentDF['E_Date']=pd.to_datetime(studentDF['End Date'], format='%d-%m-%Y')

In [6]:
from pandasql import PandaSQL
pdsql = PandaSQL()
q1 = '''select Student_ID, Institute_Category, Institute_location ,hometown ,Degree,
Stream, Current_year, Year_of_graduation, Performance_PG, PG_scale,
Performance_UG, UG_Scale, Performance_12th, Performance_10th, Experience_Type,
Profile, Location, S_Date, E_Date, SUM(Num_Exp) as Num_Exp_Row From studentDF Group BY Student_ID;'''

studentDF2 = pdsql(q1)


In [7]:
len(studentDF2.index)

72621

In [8]:
studentDF2['Stream'][studentDF2['Degree'].isnull() == True]

13287                                 Commence
24693    Fashion Lifestyle Business Management
28686                                 Commerce
31379                               Management
37776                                     None
39461                                     None
68583                                     None
Name: Stream, dtype: object

In [9]:
trainDF['tag']='train'
testDF['tag']='test'

In [10]:
testDF['Is_Shortlisted']=0
data = trainDF.append(testDF)

In [11]:
#combining data and student/intern
data1=data.merge(right=studentDF2, how='inner',on=['Student_ID'], suffixes=['','_student'])

In [12]:
intern1 = internDF[internDF.columns[0:13]]

In [13]:
data2=data1.merge(right=intern1, how='inner',on=['Internship_ID'], suffixes=['','_intern'])

In [14]:
data2['Degree'].isnull().sum()

17

In [15]:
from dateutil import parser

def normalizedate(i):
    d = parser.parse(i)
    return d.strftime("%d-%m-%Y")

data2['Earliest_Start_Date'] = data2['Earliest_Start_Date'].apply(normalizedate)
data2['Earliest_Start_Date']=pd.to_datetime(data2['Earliest_Start_Date'], format='%d-%m-%Y')
data2['Start_Date']=pd.to_datetime(data2['Start_Date'], format='%d-%m-%Y')

In [16]:
trainDF['Is_Shortlisted'].value_counts()

0    168003
1     24579
Name: Is_Shortlisted, dtype: int64

In [17]:
data2['Degree'] = data2['Degree'].astype('category')
data2['Stream'] = data2['Stream'].astype('category')
data2['Profile'] = data2['Profile'].astype('category')


In [18]:
data3 = data2.reset_index()

In [19]:
# missing value treatment of data3['Preferred_location']
# Lets tag it as No_Pref and create a feature to tag it
data3['Preferred_location'] = data3['Preferred_location'].fillna('No_Pref')
data3['Preferred_location'] = data3['Preferred_location'].astype('category')

In [20]:
data3['Degree'][(data3['Stream']=='Management')].isnull().sum()

3

In [21]:
# substituting NA values of Degree with most common category
data3['Degree'] = np.where(data3['Stream']=='Management' , data3['Degree'].fillna('MBA'), data3['Degree'])
data3['Degree'] = np.where(data3['Stream']=='Fashion Lifestyle Business Management' , data3['Degree'].fillna('MBA'), data3['Degree'])
data3['Degree'] = np.where(data3['Stream']=='Commence' , data3['Degree'].fillna('B.Com'), data3['Degree'])
data3['Degree'] = np.where(data3['Stream']=='Commerce' , data3['Degree'].fillna('B.Com'), data3['Degree'])

In [22]:
data3['Degree'] = np.where(data3['Internship_Profile']=='Design' , data3['Degree'].fillna('Designing'), data3['Degree'])
data3['Degree'] = np.where(data3['Internship_Profile']=='Social Media Marketing' , data3['Degree'].fillna('Digital Marketing'), data3['Degree'])
data3['Degree'] = np.where(data3['Internship_Profile']=='Digital Marketing' , data3['Degree'].fillna('Digital Marketing'), data3['Degree'])
data3['Degree'] = np.where(data3['Internship_Profile']=='Graphic Design' , data3['Degree'].fillna('Graphic Design'), data3['Degree'])
data3['Degree'] = np.where(data3['Internship_Profile']=='Illustration' , data3['Degree'].fillna('B.A.(Hons) Journalism'), data3['Degree'])
data3['Degree'] = np.where(data3['Internship_Profile']=='Google Ad Word Management' , data3['Degree'].fillna('MBA'), data3['Degree'])
data3['Degree'] = np.where(data3['Internship_Profile']=='Operations- Quality Analyst' , data3['Degree'].fillna('Global Business Operations (GBO)'), data3['Degree'])

In [23]:
# substituting NA values of Stream 
data3['Stream'] = np.where(data3['Degree']=='Designing' , data3['Stream'].fillna('Accessory Designing'), data3['Stream'])
data3['Stream'] = np.where(data3['Degree']=='MCA' , data3['Stream'].fillna('Computer Application'), data3['Stream'])
data3['Stream'] = np.where(data3['Degree']=='Post Graduate Dimploma in Management' , data3['Stream'].fillna('Marketing'), data3['Stream'])
data3['Stream'] = np.where(data3['Degree']=='MBA' , data3['Stream'].fillna('Marketing'), data3['Stream'])
data3['Stream'] = np.where(data3['Degree']=='B.Com (Hons.)' , data3['Stream'].fillna('Commerce'), data3['Stream'])
data3['Stream'] = np.where(data3['Degree']=='Graphic Design' , data3['Stream'].fillna('Visual Comm.'), data3['Stream'])
data3['Stream'] = np.where(data3['Degree']=='Bachelor of Business Admininstration' , data3['Stream'].fillna('Management'), data3['Stream'])
data3['Stream'] = np.where(data3['Degree']=='Digital Marketing' , data3['Stream'].fillna('Commerce'), data3['Stream'])
data3['Stream'] = np.where(data3['Degree']=='B.M.M.' , data3['Stream'].fillna('Arts'), data3['Stream'])
data3['Stream'] = np.where(data3['Degree']=='BCA' , data3['Stream'].fillna('Computer Application'), data3['Stream'])
data3['Stream'] = np.where(data3['Degree']=='Global Business Operations (GBO)' , data3['Stream'].fillna('Finance'), data3['Stream'])
data3['Stream'] = np.where(data3['Degree']=='B.A.LL.B. (Hons.)' , data3['Stream'].fillna('Law'), data3['Stream'])
data3['Stream'] = np.where(data3['Degree']=='Under' , data3['Stream'].fillna('Under'), data3['Stream'])
data3['Stream'] = np.where(data3['Degree']=='B.A. Programme' , data3['Stream'].fillna('Arts'), data3['Stream'])
data3['Stream'] = np.where(data3['Degree']=='B.Sc (Hons.) Computer Science' , data3['Stream'].fillna('Science'), data3['Stream'])
data3['Stream'] = np.where(data3['Degree']=='B.S. & M.S. (Dual)' , data3['Stream'].fillna('Mathematics and Computing'), data3['Stream'])
data3['Stream'] = np.where(data3['Degree']=='Undecided' , data3['Stream'].fillna('Undecided'), data3['Stream'])
data3['Stream'] = np.where(data3['Degree']=='B.A.(Hons) Journalism' , data3['Stream'].fillna('Arts'), data3['Stream'])
data3['Stream'] = np.where(data3['Degree']=='LLB' , data3['Stream'].fillna('Law'), data3['Stream'])

In [24]:
data3['Experience_Type'].value_counts(dropna=False)

NaN                 101713
internship           51116
academic_project     45311
training             25281
job                  18943
por                  16368
award                14953
participation        14205
workshop             10799
other                 1321
Name: Experience_Type, dtype: int64

In [25]:
data3['Profile'] = np.where(data3['Experience_Type'].isnull() == False , data3['Profile'].fillna('Intern'), data3['Profile'])
data3['Profile'] = data3['Profile'].fillna('No_Exp')
data3['Experience_Type'] = data3['Experience_Type'].fillna('No_Exp')

In [26]:
data3['Profile'] = data3['Profile'].astype('category')
data3['Experience_Type'] = data3['Experience_Type'].astype('category')

In [27]:
#NAs in S_Date , E_Date
data3['S_Date'] = np.where(data3['Experience_Type'] == 'No_Exp', data3['S_Date'].fillna('2015-02-21'), data3['S_Date'])
data3['E_Date'] = np.where(data3['Experience_Type'] == 'No_Exp', data3['E_Date'].fillna('2015-02-21'), data3['E_Date'])
data3['E_Date'] = data3['E_Date'].fillna('2015-02-21')

In [28]:
data3['E_Date'] = data3['E_Date'].apply(normalizedate)
data3['S_Date'] = data3['S_Date'].apply(normalizedate)
data3['S_Date'] = pd.to_datetime(data3['S_Date'], format='%d-%m-%Y')
data3['E_Date'] = pd.to_datetime(data3['E_Date'], format='%d-%m-%Y')

In [29]:
data3['Stipend1'] = np.where(data3['Stipend_Type'] == 'unpaid', 0, data3['Stipend1'])
## NA values replaced by median
data3['Stipend2'] = data3['Stipend2'].fillna(10000)

In [30]:
## Capping outliersin data3$Stipend1
data3['Stipend1'] = np.where(data3['Stipend1']==50000, 5000, data3['Stipend1'])
data3['Stipend1'] = np.where(data3['Stipend1']==40000, 4000, data3['Stipend1'])
data3['Stipend1'] = np.where(data3['Stipend1']==35000, 3500, data3['Stipend1'])
data3['Stipend1'] = np.where((data3['Stipend1']<=30000) & (data3['Stipend1']>=10000), 3000, data3['Stipend1'])

In [31]:
## Capping outliersin data3$Stipend2
data3['Stipend2'] = np.where(data3['Stipend2']==150000, 15000, data3['Stipend2'])

In [32]:
# Outliers in data2$Internship_Duration.Months.
data3['Internship_Duration(Months)'] = np.where(data3['Internship_Duration(Months)']==2016, 24, data3['Internship_Duration(Months)'])
data3['Internship_Duration(Months)'] = np.where(data3['Internship_Duration(Months)']==10000, 10, data3['Internship_Duration(Months)'])
data3['Internship_Duration(Months)'] = np.where(data3['Internship_Duration(Months)']==20160201, 12, data3['Internship_Duration(Months)'])
data3['Internship_Duration(Months)'] = np.where(data3['Internship_Duration(Months)']==20160331, 15, data3['Internship_Duration(Months)'])

In [33]:
# why min=0 in summary(data3$Performance_PG),summary(data3$Performance_UG),summary(data3$Performance_12th)
data3['Performance_10th'] = np.where(data3['Performance_10th']<=10, data3['Performance_10th']*10,data3['Performance_10th'])
data3['Performance_10th'] = np.where(data3['Performance_10th']<=8.5, data3['Performance_10th']*10,data3['Performance_10th'])
data3['Performance_10th'] = np.where(data3['Performance_10th']<=40, 40,data3['Performance_10th'])

In [34]:
data3['Performance_12th'] = np.where(data3['Performance_12th']<=10, data3['Performance_12th']*10,data3['Performance_12th'])
data3['Performance_12th'] = np.where(data3['Performance_12th']<=10, data3['Performance_12th']*10,data3['Performance_12th'])
data3['Performance_12th'] = np.where(data3['Performance_12th']<=40, 40,data3['Performance_12th'])

In [35]:
data3['Per_UG'] = (data3['Performance_UG']/data3['UG_Scale'])*100
data3['Per_UG'] = np.where(data3['Per_UG'] <=10, (data3['Per_UG']*10),data3['Per_UG'])
data3['Per_UG'] = np.where(data3['Per_UG'] <40, 40,data3['Per_UG'])
data3['Performance_UG'] = data3['Per_UG']

In [36]:
## Since PG_Scale is there, lets convert to ratio
data3['Per_PG'] = (data3['Performance_PG']/data3['PG_scale'])*100
data3['Per_PG'] = np.where(data3['Per_PG'] <=10, (data3['Per_PG']*10),data3['Per_PG'])
data3['Per_PG'] = np.where(data3['Per_PG'] <40, 40,data3['Per_PG'])
data3['Performance_PG'] = data3['Per_PG']

In [37]:
# Skills_required NULL
data3['Skills_required'] = data3['Skills_required'].fillna('No_Skill')

In [38]:
#Feature Engineering
data3['Exp_tenure']=0
data3['Exp_tenure'] = data3['E_Date'] - data3['S_Date']
data3['Exp_tenure'] = pd.to_numeric(data3['Exp_tenure'])
data3['Exp_tenure'] = np.where(data3['Exp_tenure'] < 30, 0, data3['Exp_tenure'])

In [39]:
data4 = data3.drop(['Per_UG','Per_PG'],axis=1)

In [40]:
## Tagging on Preferred_location
data4['Is_PlNo_Pref'] = np.where(data4['Preferred_location'] =='No_Pref', 1,0)
data4['Is_PlIHFG'] = np.where(data4['Preferred_location'] =='IHFG', 1,0)
data4['Is_PlIHJB'] = np.where(data4['Preferred_location'] =='IHJB', 1,0)
data4['Is_PlIIBD'] = np.where(data4['Preferred_location'] =='IIBD', 1,0)
data4['Is_PlIIDB'] = np.where(data4['Preferred_location'] =='IIDB', 1,0)
data4['Is_PlIJBG'] = np.where(data4['Preferred_location'] =='IJBG', 1,0)
data4['Is_PlIJCE'] = np.where(data4['Preferred_location'] =='IJCE', 1,0)
data4['Is_PlIJJI'] = np.where(data4['Preferred_location'] =='IJJI', 1,0)
data4['Is_PlJABD'] = np.where(data4['Preferred_location'] =='JABD', 1,0)
data4['Is_PlJBDB'] = np.where(data4['Preferred_location'] =='JBDB', 1,0)

In [41]:
data4['Is_InstLoc_IHHF'] = np.where(data4['Institute_location'] == 'IHHF',1,0)
data4['Is_InstLoc_IHHH'] = np.where(data4['Institute_location'] == 'IHHH',1,0)
data4['Is_InstLoc_IHJB'] = np.where(data4['Institute_location'] == 'IHJB',1,0)
data4['Is_InstLoc_IJCE'] = np.where(data4['Institute_location'] == 'IJCE',1,0)
data4['Is_InstLoc_IHJC'] = np.where(data4['Institute_location'] == 'IHJC',1,0)
data4['Is_InstLoc_IIBD'] = np.where(data4['Institute_location'] == 'IIBD',1,0)
data4['Is_InstLoc_IIDB'] = np.where(data4['Institute_location'] == 'IIDB',1,0)
data4['Is_InstLoc_IIGE'] = np.where(data4['Institute_location'] == 'IIGE',1,0)
data4['Is_InstLoc_IIIF'] = np.where(data4['Institute_location'] == 'IIIF',1,0)
data4['Is_InstLoc_IIJJ'] = np.where(data4['Institute_location'] == 'IIJJ',1,0)
data4['Is_InstLoc_IJAB'] = np.where(data4['Institute_location'] == 'IJAB',1,0)
data4['Is_InstLoc_IJAE'] = np.where(data4['Institute_location'] == 'IJAE',1,0)
data4['Is_InstLoc_IJGB'] = np.where(data4['Institute_location'] == 'IJGB',1,0)
data4['Is_InstLoc_IJBG'] = np.where(data4['Institute_location'] == 'IJBG',1,0)

In [42]:
data4['Inf_hometown'] = np.where(data4['hometown'].isin(["IHGI","IHHH","IHJB","IHJC","IIAI","IIBD","IIDB","IIGA",
                        "IIIF","IJAB","IJAE","IJBG","IJCE","IJHA","IJIG","IJJI","JAAJ",
                        "JABD","JADD","JADH","JAGD","JAHG","JBBE","JBDB","JBEB","JBEI","JBID",
                        "JCBC","JCDD","JCHJ","JDAE","JDFA","JECD","JEEH","JEHI"]),1,0)

In [43]:
## Degree 
data4['Is_BTech'] = np.where(data4['Degree'] == 'B.Tech',1,0)
data4['Is_BE'] = np.where(data4['Degree'] == 'B.E',1,0)
data4['Is_MCA '] = np.where(data4['Degree'] == 'MCA',1,0)
data4['Is_MBA'] = np.where(data4['Degree'] == 'MBA',1,0)
data4['Is_BCom'] = np.where(data4['Degree'] == 'B.Com',1,0)
data4['Is_PGDM'] = np.where(data4['Degree'] == 'Post Graduate Dimploma in Management',1,0)
data4['Is_BSc'] = np.where(data4['Degree'] == 'B.Sc',1,0)
data4['Is_BBA'] = np.where(data4['Degree'] == 'Bachelor of Business Admininstration',1,0)
data4['Is_MTech'] = np.where(data4['Degree'] == 'M.Tech',1,0)

In [44]:
## Stream
data4['Is_StrCSE'] = np.where(data4['Stream'] == 'Computer Science & Engineering',1,0)
data4['Is_StrCS'] = np.where(data4['Stream'] == 'Computer  Science',1,0)
data4['Is_StrECE'] = np.where(data4['Stream'] == 'Electronics and Communication Engineering',1,0)
data4['Is_StrCoAp'] = np.where(data4['Stream'] == 'Computer Application',1,0)
data4['Is_StrCommerce'] = np.where(data4['Stream'] == 'Commerce',1,0)
data4['Is_StrIT'] = np.where(data4['Stream'] == 'Information Technology',1,0)
data4['Is_StrME'] = np.where(data4['Stream'] == 'Mechanical Engineering',1,0)
data4['Is_StrMarketing'] = np.where(data4['Stream'] == 'Marketing',1,0)

In [45]:
#Profile
data4['Is_Prof_intern'] = np.where(data4['Profile'] == 'Intern',1,0)
data4['Is_Prof_No_Exp'] = np.where(data4['Profile'] == 'No_Exp',1,0)
data4['Is_Prof_Marketing'] = np.where((data4['Profile'] == 'Content Writing & Social Media Marketing') | (data4['Profile'] == 'Marketing'),1,0)
data4['Is_Prof_Content'] = np.where((data4['Profile'] == 'Content Writer') | (data4['Profile'] == 'Content Development'),1,0)

In [46]:
##  Location
data4['Is_LocatIIGB'] = np.where(data4['Location'] == 'IIGB',1,0)
data4['Is_LocatIIDB'] = np.where(data4['Location'] == 'IIDB',1,0)
data4['Is_LocatJEJJ'] = np.where(data4['Location'] == 'JEJJ',1,0)
data4['Is_LocatIIBD'] = np.where(data4['Location'] == 'IIBD',1,0)
data4['Is_LocatJABD'] = np.where(data4['Location'] == 'JABD',1,0)

In [47]:
## Internship_Profile
data4['Is_IP_WD'] = np.where(data4['Internship_Profile'] == 'Web Development',1,0)
data4['Is_IP_SD'] = np.where(data4['Internship_Profile'] == 'Software Development',1,0)
data4['Is_IP_CW'] = np.where(data4['Internship_Profile'] == 'Content Writing',1,0)
data4['Is_IP_AD'] = np.where(data4['Internship_Profile'] == 'Android App Development',1,0)
data4['Is_IP_MK'] = np.where(data4['Internship_Profile'] == 'Marketing',1,0)
data4['Is_IP_BD'] = np.where(data4['Internship_Profile'] == 'Business Development',1,0)

In [48]:
data4['Is_SR_No'] = np.where(data4['Skills_required'] == 'No_Skill',1,0)

In [49]:
## Internship_Location
data4['Is_IntrnLoc_IIDB'] = np.where(data4['Internship_Location'] == 'IIDB',1,0)
data4['Is_IntrnLoc_IIBD'] = np.where(data4['Internship_Location'] == 'IIBD',1,0)
data4['Is_IntrnLoc_IIGB'] = np.where(data4['Internship_Location'] == 'IIGB',1,0)
data4['Is_IntrnLoc_JABD'] = np.where(data4['Internship_Location'] == 'JABD',1,0)
data4['Is_IntrnLoc_JEJJ'] = np.where(data4['Internship_Location'] == 'JEJJ',1,0)

In [50]:
data4['Internship_deadline'] = pd.to_datetime(data4['Internship_deadline'], format='%d-%m-%Y')
dummies = pd.get_dummies(data4[['Current_year','Experience_Type','Internship_Type','Internship_category','Stipend_Type']])
data4 = data4.drop(['Current_year','Experience_Type','Internship_Type','Internship_category','Stipend_Type'], axis=1)
data4 = pd.concat([data4, dummies], axis=1)

In [51]:
# Match/ Distance between Preferred_location and Internship_Location
data4['Pref_Intern_LocMatch']=0
data4['Pref_Intern_LocMatch'] = np.where((data4['Preferred_location'] == data4['Internship_Location']) | (data4['Preferred_location'] == 'No_Pref'),1,0)

In [52]:
# Expected_Stipend (expected by student) Stipend1(min offered) Stipend2(max offered)
# Substituting Middle value of Expected_Stipend
data4['Expected_Stipend'] = np.where(data4['Expected_Stipend']=='10K+',10000,data4['Expected_Stipend'])
data4['Expected_Stipend'] = np.where(data4['Expected_Stipend']=='2-5K',3500,data4['Expected_Stipend'])
data4['Expected_Stipend'] = np.where(data4['Expected_Stipend']=='5-10K',7500,data4['Expected_Stipend'])
data4['Expected_Stipend'] = np.where(data4['Expected_Stipend']=='No Expectations',0,data4['Expected_Stipend'])
data4['Expected_Stipend'] = pd.to_numeric(data4['Expected_Stipend'])

In [53]:
# creating  Feature whether Expected_Stipend < Stipend1
data4['St_EMatch'] = np.where(data4['Expected_Stipend'] < data4['Stipend1'], 1, 0)

In [54]:
# Creating Feature about range of Stipend Offered
#Stipend2 - Stipend1
data4['Stip_range'] = abs(data4['Stipend2']-data4['Stipend1'])

In [55]:
# Creating feature Minimum_Duration is less than Internship_Duration.Months.
data4['Duration_Match'] =0
data4['Duration_Match'] = np.where(data4['Minimum_Duration'] >= data4['Internship_Duration(Months)'],1,0)

In [56]:
#Creating Feature whether there is a match between Institute_location and Internship_Location
data4['Inst_Intern_LocMatch'] = 0
data4['Inst_Intern_LocMatch'] = np.where(data4['Institute_location'] == data4['Internship_Location'],1,0)

In [57]:
#Creating Feature whether there is a match between hometown and Internship_Location
data4['hometown_Intern_LocMatch'] =0
data4['hometown_Intern_LocMatch'] = np.where(data4['hometown'] == data4['Internship_Location'], 1, 0)

In [58]:
# Creating feature difference between Year_of_graduation and year of Internship_deadline
data4['Dif_Yog_IntD'] = data4['Year_of_graduation'] - data4['Internship_deadline'].dt.year
data4['Neg_Dif_Yog_IntD'] = np.where(data4['Dif_Yog_IntD'] > 0,1,0)

In [59]:
# tagging whether a candidate  is PG 
data4['Is_PG'] = np.where((data4['Degree'].str[0] == 'M') | (data4['Degree'].str[0] == 'P'),1,0)
data4['Is_PG'] = np.where((data4['Degree'].str.contains('B.E. & MBA') | 
                 data4['Degree'].str.contains('B.Tech and M.Tech') | 
                 data4['Degree'].str.contains('Integrated')), 1, 0)

In [60]:
# tagging whether a candidate have Prof degree
data4['Is_Prof'] = np.where((data4['Degree'].str.contains('Tech')) | (data4['Degree'].str.contains('B.E')) | 
        (data4['Degree'].str.contains('MCA')) | (data4['Degree'].str.contains('MBA')) |
        (data4['Degree'].str.contains('Management')) | (data4['Degree'].str.contains('Admininstration')) |
        (data4['Degree'].str.contains('Technology')) | (data4['Degree'].str.contains('Computer')),1,0)

In [61]:
##Creating Feature whether there is a match between Location (Location of work experience) and Internship_Location
data4['Workex_Intern_LocMatch'] = np.where(data4['Location'] == data4['Internship_Location'], 1, 0)

In [62]:
# No_of_openings
# group by Internship_ID the train file to check how many applicants
# ratio of applicant to opening
RATO = pd.DataFrame(data4['Internship_ID'])
RATO['Num'] = 1
RATO1 = pdsql('''select Internship_ID, SUM(Num) as Num_Applicant From RATO Group BY Internship_ID''')

data4=data4.merge(right=RATO1, how='inner',on=['Internship_ID'], suffixes=['','_intern1'])

In [63]:
data4['Open_App_Ratio'] = data4['No_of_openings']/data4['Num_Applicant']

In [64]:
## any relation between Internship_deadline,Earliest_Start_Date
data4['Diff_Intdl_StrD'] = data4['Internship_deadline'] - data4['Earliest_Start_Date']
data4['Diff_Intdl_StrD'] = pd.to_numeric(data4['Diff_Intdl_StrD'])

In [65]:
## If applicant available before internship deadline
data4['NoCross_Deadline'] = np.where(data4['Diff_Intdl_StrD'] >0,1,0)

In [66]:
data4['Deadline2015'] = np.where(data4['Internship_deadline']> '2015-01-13',1,0)

In [67]:
#Institute_Category
data4['Institute_Category'] = np.where(data4['Institute_Category']=='Y',1,0)

In [68]:
data5 = data4.ix[:,[1,2,3,4,5,7,8,9,10,16,18,20,21,26,30,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135]]

In [69]:
## Splitting to Train and Test
Train = data5[data5['tag']=='train']
Test = data5[data5['tag']=='test']

In [70]:
Train = Train.drop('tag',axis=1)
Test = Test.drop('tag',axis=1)
Test = Test.drop('Is_Shortlisted', axis=1)

In [71]:
Train.to_csv(data_path+'TrainD.csv',index=False)
Test.to_csv(data_path+'TestD.csv',index=False)